Questions:
1. What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)
2. What is the edit distance of the best match between pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)
3. In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. The function is copied below.
```
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
```
Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least 
k bases long. To make an overlap graph, we could call overlap(a, b, min_length=k) on every possible pair of reads from the dataset.  Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read a whose length-6 suffix is GTCCTA.  Say GTCCTA does not occur in any other read in the dataset.  In other words, the 6-mer 
GTCCTA occurs at the end of read a and nowhere else.  It follows that a's suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read a and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of 
a.  This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea.  You don't have to do it this way, but this might help you.  Let every k-mer in the dataset have an associated 
Python set object, which starts out empty.  We use a Python dictionary to associate each k-mer with its corresponding set. 
(1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer.  If our read is GATTA and k=3, we would add GATTA to the set objects for 
GAT, ATT and TTA.  We do this for every read so that, at the end, each set contains all reads containing the corresponding k-mer.  
(2) Now, for each read a, we find all overlaps involving a suffix of a.  To do this, we take a's length-k suffix, find all reads containing that k-mer (obtained from the corresponding 
set) and call overlap(a, b, min_length=k) for each.

The most important point is that we do not call overlap(a, b, min_length=k) if b does not contain the length-k suffix of a.

Download and parse the read sequences from the provided Phi-X FASTQ file. We'll just use their base sequences, so you can ignore read names and base qualities.  Also, no two reads in the FASTQ have the same sequence of bases.  This makes things simpler.

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.

Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that.  (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.

Hint 2: Remember not to overlap a read with itself. If you do, your answers will be too high.

Hint 3: You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called 
overlap(a, b, min_length=k) on every pair of reads.  We also have provided a couple examples you can check against.

Picture the overlap graph corresponding to the overlaps just calculated.  How many edges are in the graph?  In other words, how many distinct pairs of reads overlap?

4. Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge?  (In other words, how many reads have a suffix involved in an overlap?)

In [96]:
# Read genome from FASTA file in folder (chr1.GRCh38.excerpt.fasta) and prepare pattern matching variables

# Set FASTA filename
filename = 'chr1.GRCh38.excerpt.fasta'

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

# Set text 't' to the genome from file
t = readGenome(filename)

# Specify shorter text 't' for testing
# t = 'TATTGGCTATACGGTT'

# Set pattern 'p' to desired string to match
p = 'GCTGATCGATCGTACG'

In [97]:
# Read sequences and qualities from imported FASTQ file

# Set FASTQ filename
filename = 'ERR266411_1.for_asm.fastq'

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename, 'r') as fq:
        while True:
            fq.readline()  # skip name line
            seq = fq.readline().rstrip()  # read base sequence
            fq.readline()  # skip placeholder line
            qual = fq.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

sequences, qualities = readFastq(filename)

In [98]:
pattern_len = len(p)
min_dist = None

def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

for i in range(len(t) - pattern_len + 1):
    substring = t[i:i+pattern_len]
    dist = editDistance(p, substring)
    if (min_dist is None) or (dist < min_dist):
        min_dist = dist

print(min_dist)

3


In [99]:
def read_fastq(filename):
    sequences = []
    with open(filename, 'r') as fh:
        while True:
            fh.readline()  # name line
            seq = fh.readline().rstrip()  # sequence
            fh.readline()  # plus line
            fh.readline()  # quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

# Load reads
reads = read_fastq("ERR266411_1.for_asm.fastq")
print(f"Total reads loaded: {len(reads)}")

def overlap(a, b, min_length=30):
    """Return length of longest suffix of 'a' matching prefix of 'b' with len ≥ min_length."""
    start = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a) - start
        start += 1

from collections import defaultdict

def build_kmer_index(reads, k):
    index = defaultdict(set)
    for read in reads:
        for i in range(len(read) - k + 1):
            kmer = read[i:i + k]
            index[kmer].add(read)
    return index

def find_all_overlaps(reads, k=30):
    index = build_kmer_index(reads, k)
    overlaps = {}
    for a in reads:
        suffix = a[-k:]
        candidates = index.get(suffix, set())
        for b in candidates:
            if a != b:
                olen = overlap(a, b, k)
                if olen >= k:
                    overlaps[(a, b)] = olen
    return overlaps

print(f"Total overlaps found: {len(overlaps)}")

# Count reads with at least one outgoing edge
outgoing = set(a for (a, b) in overlaps)
print(f"Reads with at least one outgoing edge: {len(outgoing)}")

Total reads loaded: 10000
Total overlaps found: 27044
Reads with at least one outgoing edge: 4353


In [2]:
from collections import defaultdict

# Function to read FASTQ sequences
def readFastq(filename):
    sequences = []
    with open(filename, 'r') as fq:
        while True:
            fq.readline()  # skip header
            seq = fq.readline().rstrip()
            fq.readline()  # skip plus line
            fq.readline()  # skip quality line
            if not seq:
                break
            sequences.append(seq)
    return sequences

# Overlap function (longest suffix of a matching prefix of b >= min_length)
def overlap(a, b, min_length=30):
    start = 0
    max_olen = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            break
        olen = len(a) - start
        if olen >= min_length and b.startswith(a[start:]):
            max_olen = max(max_olen, olen)
        start += 1
    return max_olen

def find_overlaps(reads, k=30):
    # Deduplicate reads
    reads_unique = list(set(reads))

    # Build prefix index: prefix_kmer -> set of read indices
    prefix_index = defaultdict(set)
    for i, read in enumerate(reads_unique):
        prefix = read[:k]
        prefix_index[prefix].add(i)

    overlaps = {}
    for i, read_a in enumerate(reads_unique):
        suffix = read_a[-k:]
        candidates = prefix_index.get(suffix, set())
        for j in candidates:
            if i != j:
                read_b = reads_unique[j]
                olen = overlap(read_a, read_b, min_length=k)
                if olen > 0:
                    overlaps[(i, j)] = olen

    return overlaps, reads_unique

# === Main execution ===
filename = 'ERR266411_1.for_asm.fastq'  # make sure this file is in the working dir
reads = readFastq(filename)

overlap_map, unique_reads = find_overlaps(reads, k=30)

print(f"Total reads loaded: {len(reads)}")
print(f"Unique reads after deduplication: {len(unique_reads)}")
print(f"Total overlaps found (length >= 30): {len(overlap_map)}")

outgoing_nodes = set(i for i, j in overlap_map.keys())
print(f"Reads with at least one outgoing overlap: {len(outgoing_nodes)}")


Total reads loaded: 10000
Unique reads after deduplication: 10000
Total overlaps found (length >= 30): 27044
Reads with at least one outgoing overlap: 4353
